# Statistical Validation with SciTeX

This notebook demonstrates the statistical validation framework in SciTeX, which helps ensure the scientific validity of your statistical analyses.

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import matplotlib.pyplot as plt
from scitex.stats import StatisticalValidator, EffectSizeCalculator
import scitex as stx

# Set random seed for reproducibility
np.random.seed(42)

## 1. Checking Statistical Assumptions

Before running statistical tests, it's crucial to check that your data meets the test assumptions.

### 1.1 Normality Testing

In [ ]:
# Generate example data
normal_data = np.random.normal(loc=100, scale=15, size=100)
skewed_data = np.random.exponential(scale=10, size=100)

# Check normality
is_normal, p_value, stats = StatisticalValidator.check_normality(normal_data)
print("Normal distribution:")
print(f"  Normally distributed: {is_normal}")
print(f"  P-value: {p_value:.4f}")
print(f"  Skewness: {stats['skew']:.3f}")
print(f"  Kurtosis: {stats['kurtosis']:.3f}\n")

is_normal, p_value, stats = StatisticalValidator.check_normality(skewed_data)
print("Exponential distribution:")
print(f"  Normally distributed: {is_normal}")
print(f"  P-value: {p_value:.4f}")
print(f"  Skewness: {stats['skew']:.3f}")
print(f"  Kurtosis: {stats['kurtosis']:.3f}")

In [ ]:
# Visualize the distributions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(normal_data, bins=20, density=True, alpha=0.7, edgecolor='black')
ax1.set_title('Normal Distribution')
ax1.set_xlabel('Value')
ax1.set_ylabel('Density')

ax2.hist(skewed_data, bins=20, density=True, alpha=0.7, edgecolor='black')
ax2.set_title('Exponential Distribution (Skewed)')
ax2.set_xlabel('Value')
ax2.set_ylabel('Density')

plt.tight_layout()
plt.show()

### 1.2 Homoscedasticity Testing (Equal Variances)

In [ ]:
# Groups with equal variances
group1_equal = np.random.normal(100, 10, 50)
group2_equal = np.random.normal(105, 11, 50)

# Groups with unequal variances
group1_unequal = np.random.normal(100, 10, 50)
group2_unequal = np.random.normal(105, 30, 50)

# Test equal variances
is_homo, p_value, stats = StatisticalValidator.check_homoscedasticity(group1_equal, group2_equal)
print("Groups with similar variances:")
print(f"  Homoscedastic: {is_homo}")
print(f"  P-value: {p_value:.4f}")
print(f"  Variance ratio: {stats['variance_ratio']:.2f}\n")

# Test unequal variances
is_homo, p_value, stats = StatisticalValidator.check_homoscedasticity(group1_unequal, group2_unequal)
print("Groups with different variances:")
print(f"  Homoscedastic: {is_homo}")
print(f"  P-value: {p_value:.4f}")
print(f"  Variance ratio: {stats['variance_ratio']:.2f}")

### 1.3 Sample Size Validation

In [ ]:
# Check sample size for different tests
small_sample = np.random.normal(100, 15, 20)
medium_sample = np.random.normal(100, 15, 50)

# For t-test
is_adequate, info = StatisticalValidator.validate_sample_size(small_sample, 't_test')
print(f"T-test with n={len(small_sample)}:")
print(f"  Adequate: {is_adequate}")
print(f"  Recommendation: {info.get('power_recommendation', 'Sample size OK')}\n")

# For correlation
is_adequate, info = StatisticalValidator.validate_sample_size(medium_sample, 'correlation')
print(f"Correlation with n={len(medium_sample)}:")
print(f"  Adequate: {is_adequate}")
print(f"  Recommendation: {info.get('power_recommendation', 'Sample size OK')}")

## 2. Choosing the Right Statistical Test

The validator can suggest appropriate tests based on your data characteristics.

In [ ]:
# Example 1: Normal data with equal variances
data_chars = {
    'is_normal': True,
    'is_homoscedastic': True,
    'n_groups': 2,
    'is_paired': False,
    'sample_size': 50
}

suggestions = StatisticalValidator.suggest_test(data_chars, 'two_sample')
print("Scenario 1: Normal data, equal variances")
print(f"  Recommended test: {suggestions['primary']}")
print(f"  Alternatives: {suggestions['alternatives']}\n")

# Example 2: Non-normal data
data_chars['is_normal'] = False
suggestions = StatisticalValidator.suggest_test(data_chars, 'two_sample')
print("Scenario 2: Non-normal data")
print(f"  Recommended test: {suggestions['primary']}")
print(f"  Rationale: {suggestions['rationale'][0]}\n")

# Example 3: Paired data
data_chars['is_paired'] = True
suggestions = StatisticalValidator.suggest_test(data_chars, 'two_sample')
print("Scenario 3: Paired non-normal data")
print(f"  Recommended test: {suggestions['primary']}")
print(f"  Alternatives: {suggestions['alternatives']}")

## 3. Effect Size Calculations

P-values tell you if an effect exists, but effect sizes tell you how large it is.

### 3.1 Cohen's d for Two Groups

In [ ]:
# Small effect (d ≈ 0.2)
control_small = np.random.normal(100, 15, 100)
treatment_small = np.random.normal(103, 15, 100)

# Large effect (d ≈ 0.8)
control_large = np.random.normal(100, 15, 100)
treatment_large = np.random.normal(112, 15, 100)

# Calculate effect sizes
small_effect = EffectSizeCalculator.cohens_d(treatment_small, control_small)
large_effect = EffectSizeCalculator.cohens_d(treatment_large, control_large)

print("Small effect:")
print(f"  Cohen's d = {small_effect['d']:.3f} ({small_effect['interpretation']})")
print(f"  Mean difference = {small_effect['mean_diff']:.2f}")
print(f"  95% CI: [{small_effect['ci_lower']:.3f}, {small_effect['ci_upper']:.3f}]\n")

print("Large effect:")
print(f"  Cohen's d = {large_effect['d']:.3f} ({large_effect['interpretation']})")
print(f"  Mean difference = {large_effect['mean_diff']:.2f}")
print(f"  95% CI: [{large_effect['ci_lower']:.3f}, {large_effect['ci_upper']:.3f}]")

In [ ]:
# Visualize effect sizes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Small effect
ax1.hist(control_small, bins=20, alpha=0.5, label='Control', density=True)
ax1.hist(treatment_small, bins=20, alpha=0.5, label='Treatment', density=True)
ax1.axvline(control_small.mean(), color='blue', linestyle='--')
ax1.axvline(treatment_small.mean(), color='orange', linestyle='--')
ax1.set_title(f'Small Effect (d = {small_effect["d"]:.2f})')
ax1.legend()

# Large effect
ax2.hist(control_large, bins=20, alpha=0.5, label='Control', density=True)
ax2.hist(treatment_large, bins=20, alpha=0.5, label='Treatment', density=True)
ax2.axvline(control_large.mean(), color='blue', linestyle='--')
ax2.axvline(treatment_large.mean(), color='orange', linestyle='--')
ax2.set_title(f'Large Effect (d = {large_effect["d"]:.2f})')
ax2.legend()

plt.tight_layout()
plt.show()

### 3.2 Effect Sizes for ANOVA

In [ ]:
# Three groups with medium effect
groups = [
    np.random.normal(100, 15, 50),  # Control
    np.random.normal(105, 15, 50),  # Treatment 1
    np.random.normal(110, 15, 50),  # Treatment 2
]

# Calculate eta-squared and omega-squared
eta_result = EffectSizeCalculator.eta_squared(groups)
omega_result = EffectSizeCalculator.omega_squared(groups)

print("ANOVA Effect Sizes:")
print(f"  Eta-squared = {eta_result['eta_squared']:.3f} ({eta_result['interpretation']})")
print(f"  Omega-squared = {omega_result['omega_squared']:.3f} ({omega_result['interpretation']})")
print(f"\nNote: Omega-squared is less biased for small samples")

### 3.3 Effect Sizes for Contingency Tables

In [ ]:
# 2x2 contingency table
# Rows: Treatment/Control, Columns: Success/Failure
contingency_table = np.array([
    [45, 15],  # Treatment: 45 success, 15 failure
    [30, 30]   # Control: 30 success, 30 failure
])

# Calculate odds ratio and relative risk
or_result = EffectSizeCalculator.odds_ratio(contingency_table)
rr_result = EffectSizeCalculator.relative_risk(contingency_table)

print("Contingency Table:")
print(contingency_table)
print(f"\nOdds Ratio = {or_result['odds_ratio']:.2f}")
print(f"  95% CI: [{or_result['ci_lower']:.2f}, {or_result['ci_upper']:.2f}]")
print(f"  Interpretation: {or_result['interpretation']}")
print(f"\nRelative Risk = {rr_result['relative_risk']:.2f}")
print(f"  95% CI: [{rr_result['ci_lower']:.2f}, {rr_result['ci_upper']:.2f}]")
print(f"  Interpretation: {rr_result['interpretation']}")

## 4. Practical Example: Complete Analysis Workflow

Let's walk through a complete analysis with validation.

In [ ]:
# Simulate experimental data
np.random.seed(123)

# Control group: baseline performance
control = np.random.normal(75, 12, 25)

# Treatment group: new intervention
treatment = np.random.normal(82, 15, 28)

print("Step 1: Check assumptions")
print("=" * 40)

# Check normality
control_normal, _, _ = StatisticalValidator.check_normality(control)
treatment_normal, _, _ = StatisticalValidator.check_normality(treatment)
print(f"Control group normal: {control_normal}")
print(f"Treatment group normal: {treatment_normal}")

# Check equal variances
is_homo, p_homo, _ = StatisticalValidator.check_homoscedasticity(control, treatment)
print(f"Equal variances: {is_homo} (p={p_homo:.3f})")

# Check sample size
is_adequate, size_info = StatisticalValidator.validate_sample_size([control, treatment], 't_test')
print(f"Sample size adequate: {is_adequate}")
print(f"Group sizes: {size_info['group_sizes']}")

In [ ]:
print("\nStep 2: Choose appropriate test")
print("=" * 40)

# Get test recommendation
data_characteristics = {
    'is_normal': control_normal and treatment_normal,
    'is_homoscedastic': is_homo,
    'n_groups': 2,
    'is_paired': False,
    'sample_size': min(len(control), len(treatment))
}

test_suggestion = StatisticalValidator.suggest_test(data_characteristics, 'two_sample')
print(f"Recommended test: {test_suggestion['primary']}")
print(f"Alternatives: {test_suggestion['alternatives']}")

# Run the appropriate test
from scipy import stats
if test_suggestion['primary'] == 'independent_t_test':
    stat, p_value = stats.ttest_ind(control, treatment)
    test_name = "Independent t-test"
elif test_suggestion['primary'] == 'welch_t_test':
    stat, p_value = stats.ttest_ind(control, treatment, equal_var=False)
    test_name = "Welch's t-test"
else:
    # Use SciTeX's Brunner-Munzel test
    from scitex.stats import brunner_munzel
    result = brunner_munzel(control, treatment)
    stat, p_value = result['statistic'], result['pvalue']
    test_name = "Brunner-Munzel test"

print(f"\nTest: {test_name}")
print(f"Statistic: {stat:.3f}")
print(f"P-value: {p_value:.4f}")

In [ ]:
print("\nStep 3: Calculate effect size")
print("=" * 40)

# Cohen's d
effect = EffectSizeCalculator.cohens_d(treatment, control)
print(f"Cohen's d = {effect['d']:.3f} ({effect['interpretation']})")
print(f"95% CI: [{effect['ci_lower']:.3f}, {effect['ci_upper']:.3f}]")
print(f"Mean difference: {effect['mean_diff']:.2f} points")

# Hedges' g (for smaller samples)
hedges = EffectSizeCalculator.hedges_g(treatment, control)
print(f"\nHedges' g = {hedges['g']:.3f} (bias-corrected)")

# Interpretation
print(f"\nConclusion:")
if p_value < 0.05:
    print(f"✓ Statistically significant difference (p={p_value:.4f})")
else:
    print(f"✗ No statistically significant difference (p={p_value:.4f})")
print(f"✓ Effect size is {effect['interpretation']} (d={effect['d']:.3f})")

In [ ]:
# Visualize results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Box plot
ax1.boxplot([control, treatment], labels=['Control', 'Treatment'])
ax1.set_ylabel('Score')
ax1.set_title('Group Comparison')
ax1.grid(True, alpha=0.3)

# Effect size visualization
x = np.linspace(40, 120, 1000)
control_curve = stats.norm.pdf(x, control.mean(), control.std())
treatment_curve = stats.norm.pdf(x, treatment.mean(), treatment.std())

ax2.fill_between(x, control_curve, alpha=0.3, label='Control')
ax2.fill_between(x, treatment_curve, alpha=0.3, label='Treatment')
ax2.axvline(control.mean(), color='blue', linestyle='--', alpha=0.7)
ax2.axvline(treatment.mean(), color='orange', linestyle='--', alpha=0.7)
ax2.set_xlabel('Score')
ax2.set_ylabel('Density')
ax2.set_title(f'Effect Size: d = {effect["d"]:.2f}')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Best Practices Summary

1. **Always check assumptions** before running statistical tests
2. **Report effect sizes** along with p-values
3. **Use appropriate tests** based on data characteristics
4. **Consider sample size** and statistical power
5. **Visualize your data** to understand patterns

The statistical validation framework in SciTeX helps ensure your analyses are scientifically valid and your conclusions are well-supported.